# OpenAI - REST Augmented

In [1]:
#r "nuget: dotenv.net"
#r "nuget: Newtonsoft.Json"

using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text.Json;
using dotenv.net;

#!import Models/OpenAIModels.cs

// HttpClient Instance
var client = new HttpClient();

Installed Packages dotenv.net, 3.1.2 Newtonsoft.Json, 13.0.3

## Load the API KEY and URI

In [2]:
DotEnv.Load();
var uri = Environment.GetEnvironmentVariable("GPT_OPENAI_FULL_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var deploymentName = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_ENDPOINT");
Console.WriteLine($"Using deployment at: {uri} with key {apiKey.Substring(0, 5)}...");

Using deployment at: https://alemorfr.openai.azure.com/openai/deployments/gpt/chat/completions?api-version=2023-03-15-preview with key 32c87...


## Prepare the prompt

In [3]:
// https://learn.microsoft.com/en-us/azure/aks/azure-cni-overlay

var aksNetworking = @"There are two types networking in Azure Kubernetes Service (AKS), Azure CNI overlay and Kubenet. 
CNI is well suited for 1000 nodes and 250 pods/node, there's no exatra configuration required, has higher performance, 
can work with network policies such as Calico, Cillium and Azure Network policies, and works on both Windows and Linux. 
Kubenet on the other hand, supports 400 nodes and 250 pods/node, 
has a more complex configuration, add an extra networking hop adding a minor latency, 
only supports Calico for network policies and only supports Linux.";

var template = @"What are the networking options in AKS? Which one is recommended for performance? 
Use the following text to answer the question and if you do not know the answer say, I don't know.

text: """"""
{{$input}}
""""""";
template = template.Replace("{{$input}}", aksNetworking);

var messages = new[]{new Message("user", template)};

var obj = new {
    messages=messages, 
    max_tokens=200,
    temperature=0.1
};

var json = JsonSerializer.Serialize(obj);
var content = new StringContent(json, Encoding.UTF8, "application/json");

In [4]:
var request = new HttpRequestMessage(HttpMethod.Post, uri);
request.Headers.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));
request.Headers.TryAddWithoutValidation("api-key",apiKey);
request.Content = content;

var response = await client.SendAsync(request);
if (response.IsSuccessStatusCode)
{
    var raw = await response.Content.ReadAsStringAsync();
    var completion = JsonSerializer.Deserialize<Completion>(raw);
    var responseJson = JsonDocument.Parse(raw);
    Console.WriteLine(JsonSerializer.Serialize(responseJson, new JsonSerializerOptions { WriteIndented = true }));   
    //Console.WriteLine(completion.choices[0].text);
}
else
{
    Console.WriteLine(response.StatusCode);
}

{
  "id": "chatcmpl-7iT31jHB9G6fBh9PhEwfNwS0QSMP0",
  "object": "chat.completion",
  "created": 1690832447,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "There are two networking options in AKS: Azure CNI overlay and Kubenet. Azure CNI overlay is recommended for performance as it has higher performance, can work with various network policies, and works on both Windows and Linux."
      }
    }
  ],
  "usage": {
    "completion_tokens": 46,
    "prompt_tokens": 178,
    "total_tokens": 224
  }
}
